# Ticketbox_영화매표소
- 카운터 : Resource
- 영화관 : Container
- Shared Event, Conditional Event

크리스토퍼 놀란 감독의 4편의 영화('다크나이트', '인셉션', '테넷', '인터스텔라') 티켓을 판매하는 하나의 티켓 카운터가 있는 영화관 시뮬레이션 환경을 생성합니다. 

사람들은 랜덤한 시간에 도착하여 랜덤한 영화를 위한 랜덤한 숫자의 티켓을 사려고 시도합니다. 영화가 매진되면, 그 영화의 티켓을 사기 위해 기다리는 모든 사람들이 집에 돌아갑니다. 집에 돌아가기 전에 약간의 항의를 한 후 떠납니다.

영화관은 관련된 모든 데이터(영화, 카운터, 남은 티켓, 수집된 데이터 등)를 저장하는 Container일 뿐입니다. 카운터는 용량이 1인 Resource입니다.

티켓 구매자는 자신의 차례가 될 때까지(카운터 리소스를 획득할 때까지) 또는 매진 신호가 트리거될 때까지 대기하기 시작합니다. 매진 신호가 트리거 된다면 큐에서 탈퇴, 즉 집으로 돌아갑니다. 티켓 구매자가 카운터에 도착하면, 표를 사려고 할 것입니다. 티켓 구매자들은 1장의 표만을 사지는 않습니다. 가족들 표를 위해 혹은 암표 판매를 위해 다수의 티켓을 구매할 수 있습니다. 예를 들어, 프로세스가 5장의 티켓을 구매하려고 하지만 3장만 남은 경우에는 성공하지 못할 수도 있다는 뜻입니다. 구매자가 구매한 후에 남은 티켓이 2장 미만이 된다면 매진 신호가 트리거됩니다.

In [8]:
import collections
import random
import simpy

RANDOM_SEED = 42
TICKETS = 50  # 영화에 할당되는 표의 수
SIM_TIME = 360  # 전체 시뮬레이션 시간(분)

In [9]:
def moviegoer(env, movie, num_tickets, theater):
    """티켓 구매자는 *theater*에서 특정 *movie*의 표 (*num_tickets*)개를 구매합니다.

    만약 영화가 매진되었다면 티켓 구매자는 그 즉시 매표소를 떠납니다.
    구매자가 카운터에 오면 티켓을 구매하려고 시도합니다.
    충분한 수의 티켓이 남아있지 않는다면 그는 판매원에게 항의를 한 후 떠납니다.
    
    만약 구매자의 구매 이후 티켓이 2장 미만(1장) 남게 된다면 매진 임박 신호가 트리거 됩니다.
    해당 영화에 대한 매진 임박 신호가 트리거 되면 남아있는 모든 티켓 구매자가 매표소를 떠납니다.
    """
    with theater.counter.request() as my_turn:
        # 티켓 구매자는 본인의 차례가 되거나, 영화가 매진될 때까지 줄에서 기다립니다.
        result = yield my_turn | theater.sold_out[movie]

        # 본인의 차례가 되었거나, 영호가 매진되었는지 확인합니다.
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            return

        # 영화 표가 남아있는지 확인합니다.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(0.5)
            print('티켓구매자가 %d 분에 티켓박스에 항의 후 떠납니다' % (env.now))
            return

        # 티켓을 구매합니다.
        theater.available[movie] -= num_tickets
        print('(%d분) %s 표가 %d장 판매되었습니다. %s 표는 현재 %d장 남아있습니다.' 
                % (env.now, movie, num_tickets, movie, theater.available[movie]))
        if theater.available[movie] < 2:
            # 해당 영화에 대한 매진임박 신호 트리거
            print(f'{movie} 영화 매진 임박!!!' )
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)

In [10]:
def customer_arrivals(env, theater):
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 8)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))

In [16]:
Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                 'sold_out, when_sold_out, '
                                 'num_renegers')


# 초기 설정 및 시뮬레이션 생성
print('######### 영화 매표소 시뮬레이션 시작! #############')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# 영화관 매표소 환경 생성
counter = simpy.Resource(env, capacity=1)
movies = ['다크나이트', '인셉션', '테넷', '인터스텔라'] # 영화 목록
available = {movie: TICKETS for movie in movies} # 남은 티켓 수량
sold_out = {movie: env.event() for movie in movies} # 매진 여부
when_sold_out = {movie: None for movie in movies} # 매진 시간
num_renegers = {movie: 0 for movie in movies} # 줄 서있는 사람 수
theater = Theater(counter, movies, available, sold_out, when_sold_out, num_renegers)

# 프로세스 런
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# 결과 분석
print('-----------최종 정산------------')
print('금주 상영 영화 목록 : ', movies)
for movie in movies:
    if theater.sold_out[movie]:
        print('영화 <%s> 티켓은 판매 %d 분만에 모두 판매되었습니다.'
                  % (movie, theater.when_sold_out[movie]))
        print('--- %s 명의 사람들이 줄서있다가 돌아갔네요...' %
              theater.num_renegers[movie])

######### 영화 매표소 시뮬레이션 시작! #############
(0분) 다크나이트 표가 5장 판매되었습니다. 다크나이트 표는 현재 45장 남아있습니다.
(1분) 인셉션 표가 2장 판매되었습니다. 인셉션 표는 현재 48장 남아있습니다.
(2분) 다크나이트 표가 7장 판매되었습니다. 다크나이트 표는 현재 38장 남아있습니다.
(3분) 다크나이트 표가 4장 판매되었습니다. 다크나이트 표는 현재 34장 남아있습니다.
(4분) 다크나이트 표가 4장 판매되었습니다. 다크나이트 표는 현재 30장 남아있습니다.
(5분) 인터스텔라 표가 4장 판매되었습니다. 인터스텔라 표는 현재 46장 남아있습니다.
(6분) 테넷 표가 1장 판매되었습니다. 테넷 표는 현재 49장 남아있습니다.
(7분) 인셉션 표가 7장 판매되었습니다. 인셉션 표는 현재 41장 남아있습니다.
(8분) 인셉션 표가 4장 판매되었습니다. 인셉션 표는 현재 37장 남아있습니다.
(9분) 테넷 표가 2장 판매되었습니다. 테넷 표는 현재 47장 남아있습니다.
(10분) 다크나이트 표가 6장 판매되었습니다. 다크나이트 표는 현재 24장 남아있습니다.
(11분) 테넷 표가 1장 판매되었습니다. 테넷 표는 현재 46장 남아있습니다.
(12분) 다크나이트 표가 7장 판매되었습니다. 다크나이트 표는 현재 17장 남아있습니다.
(13분) 테넷 표가 6장 판매되었습니다. 테넷 표는 현재 40장 남아있습니다.
(14분) 다크나이트 표가 1장 판매되었습니다. 다크나이트 표는 현재 16장 남아있습니다.
(15분) 테넷 표가 2장 판매되었습니다. 테넷 표는 현재 38장 남아있습니다.
(16분) 다크나이트 표가 7장 판매되었습니다. 다크나이트 표는 현재 9장 남아있습니다.
(17분) 테넷 표가 3장 판매되었습니다. 테넷 표는 현재 35장 남아있습니다.
(18분) 인셉션 표가 5장 판매되었습니다. 인셉션 표는 현재 32장 남아있습니다.
(19분) 다크나이트 표가 3장 판매되었습니다. 다크나이트 표는 현재 6장 남아있습니다.
(20